In [ ]:
%load_ext autoreload
%autoreload 2

import requests
import re
from datetime import datetime
from src.models import CentrisUrl, BienCentrisDuplex, CustomDateFormat, Utilisation
from src.utils import extract_data_from_url
from src.centris_data_parser import CentrisBienParser, Centrisdata_parser, ParsedUrl
import requests
from selectolax.parser import HTMLParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
plex_start_url="https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail&uc=1"
url_bien = "https://www.centris.ca/fr/duplex~a-vendre~montreal-mercier-hochelaga-maisonneuve/26772323?view=Summary"

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url_bien, headers=headers)

if response.status_code == 200:
    tree = HTMLParser(response.text)

In [ ]:
# inspect html
print(tree.html)

In [18]:
parsed_url = ParsedUrl(url=url_bien, tree=tree)
data_parser = CentrisBienParser(parsed_url)

In [19]:
data_parser.carac_data

{'Utilisation de la propriété': 'Résidentielle',
 'Style de bâtiment': 'En rangée',
 'Année de construction': '1958',
 'Superficie du bâtiment (au sol)': '1 000 pc',
 'Superficie habitable': '2 000 pc',
 'Superficie du terrain': '2 025 pc',
 'Stationnement total': 'Allée (1), Garage (2)',
 'Nombre d’unités': 'Résidentiel (2)',
 'Unités résidentielles': '1 x 5 ½, 1 x 9 ½',
 'Unité principale': '9 pièces, 3 chambres, 2 salles de bain',
 'Revenus bruts potentiels': '45 000 $',
 'Caractéristiques additionnelles': 'Sous-sol 6 pieds +',
 'Date d’emménagement': 'Selon les baux'}

In [ ]:
# data from URL
centris_url_data = extract_data_from_url(url_bien)
centris_id = centris_url_data['centris_id']
ville = centris_url_data['ville']
quartier = centris_url_data['quartier']

# data from HTML
price_span = data_parser.extract_price()
description = data_parser.extract_description()
adresse = data_parser.extract_address()

annee_construction = data_parser.extract_annee_construction()
superficie_terrain = data_parser.extract_superficie_terrain()
superficie_commerce = data_parser.extract_superficie_commerce()

stationnement = data_parser.extract_stationnement()
style_batiment = data_parser.extract_style_batiment()
nombre_unites = data_parser.extract_nombre_unites()
unites = data_parser.extract_unites()
revenus_brut = data_parser.extract_revenus_brut()

# Extracting municipal evaluation
eval_total_div = soup.find('tr', class_='financial-details-table-total')
if eval_total_div and eval_total_div.find('td', class_='font-weight-bold text-right'):
    eval_municipale = int(re.sub(r'[^0-9]', '', eval_total_div.find('td', class_='font-weight-bold text-right').text.strip()))

# Extracting taxes
yearly_taxes_div = soup.find('div', class_='financial-details-table-yearly')
taxes_total_div = yearly_taxes_div.find_next('tfoot').find('tr', class_='col pl-0 financial-details-table-total')
if taxes_total_div and taxes_total_div.find('td', class_='font-weight-bold text-right'):
    taxes = int(re.sub(r'[^0-9]', '', taxes_total_div.find('td', class_='font-weight-bold text-right').string.strip()))

# Extracting additional characteristics
misc_div = soup.find('div', string='Caractéristiques additionnelles')
if misc_div and misc_div.find_next_sibling('div', class_='carac-value'):
    misc = misc_div.find_next_sibling('div', class_='carac-value').string.strip()


print(f"Price: {price}")
print(f"Description: {description}")
print(f"Address: {adresse}")
print(f"Construction Year: {annee_construction}")
print(f"Land Area: {superficie_terrain}")
#print(f"Garage: {garage}")
print(f"Number of Units: {nombre_unites}")
# print(f"Residential Units: {unites}")
print(f"Municipal Evaluation: {eval_municipale}")
print(f"Taxes: {taxes}")
print(f"Potential Gross Revenue: {revenus_brut}")
# print(f"Additional Characteristics: {misc}")


Price: 749000
Description: Charmant duplex semi-commercial sur le Plateau Mont-Royal, comprenant un 2e étage vacant de 3 chambres avec terrasse spacieuse et garage, idéal pour y habiter ou louer. Le rez-de-chaussée est occupé par un dépanneur loué à long terme, offrant un revenu stable avec un bon locataire. Emplacement exceptionnel au coin de Cartier et Gilford, au coeur d'un quartier vivant et prisé. Une opportunité rare, à ne pas manquer!
Address: 4800 - 4802, Rue Cartier, Montréal (Le Plateau-Mont-Royal), Quartier Le Plateau-Mont-Royal
Construction Year: 1912
Land Area: 1677
Number of Units: 1
Municipal Evaluation: 500300
Taxes: 9033
Potential Gross Revenue: 45900


In [5]:
carac_containers = tree.css('.carac-container')

# Dictionary to store extracted data
carac_data = {}

# Iterate over each carac-container
for container in carac_containers:
    title_node = container.css_first('.carac-title')
    value_node = container.css_first('.carac-value span')
    
    if title_node and value_node:
        title = title_node.text().strip()
        value = value_node.text().strip()
        carac_data[title] = value

# Output the extracted data
for title, value in carac_data.items():
    print(f"{title}: {value}")

Utilisation de la propriété: Résidentielle, Commerciale
Style de bâtiment: En rangée
Année de construction: 1912
Superficie commerciale disponible: 1 093 pc
Superficie du terrain: 1 677 pc
Nombre d’unités: Résidentiel (1), Commercial (1)
Unité résidentielle: 1 x 8 ½
Unité principale: 8 pièces, 3 chambres, 1 salle de bain
Revenus bruts potentiels: 45 900 $
Date d’emménagement: 30 jours après l’acceptation de la promesse d’achat ou de location


In [ ]:
# Extracting municipal evaluation
eval_total_div = tree.css_first('tr.financial-details-table-total')
if eval_total_div:
    bold_cell = eval_total_div.css_first('td.font-weight-bold.text-right')
    if bold_cell and bold_cell.text():
        bien_centris['eval_municipale'] = int(re.sub(r'[^0-9]', '', bold_cell.text().strip()))

# Extracting taxes
yearly_taxes_div = tree.css_first('div.financial-details-table-yearly')
if yearly_taxes_div:
    taxes_total_div = yearly_taxes_div.css_first('tfoot tr.col.pl-0.financial-details-table-total')
    if taxes_total_div:
        bold_cell = taxes_total_div.css_first('td.font-weight-bold.text-right')
        if bold_cell and bold_cell.text():
            bien_centris['taxes'] = int(re.sub(r'[^0-9]', '', bold_cell.text().strip()))

# Extracting additional characteristics
misc_div = tree.css('div:contains("Caractéristiques additionnelles")')
if misc_div:
    sibling = misc_div[0].next_element_by_css('.carac-value')
    if sibling and sibling.text():
        bien_centris['misc'] = sibling.text().strip()

In [77]:
BienCentrisDuplex(centris_id = 1,
    url= CentrisUrl(url=url_bien),
    adresse = "",
    ville = "",
    quartier = "",
    annee_construction = 12,
    utilisation = Utilisation["Résidentiel"],
    garage = 0,
    prix = 12,
    eval_municipale = 12,
    taxes = 12,
    frais = 12,
    date_scrape = CustomDateFormat(date="2024-01-10"))

BienCentrisDuplex(centris_id=1, url=CentrisUrl(url='https://www.centris.ca/fr/duplex~a-vendre~montreal-le-plateau-mont-royal/21901946?view=Summary&uc=2'), adresse='', ville='', quartier='', annee_construction=12, utilisation=<Utilisation.Résidentiel: 'Résidentiel'>, superficie_habitable=None, superficie_batiment=None, superficie_terrain=None, garage=0, prix=12, eval_municipale=12, taxes=12, frais=12, revenus=None, description=None, date_scrape=CustomDateFormat(date='2024-01-10'))

In [ ]:
from playwright.sync_api import sync_playwright
playwright = sync_playwright().start()
browser = playwright.chromium.launch(headless=False)
page = browser.new_page()
page.goto("https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail")
page.cl(path="example.png")
browser.close()
playwright.stop()